In [31]:
import pandas as pd
import requests
import numpy as np
from requests.auth import HTTPBasicAuth
import json
import time
from tqdm import tqdm

#### Preprocess Query input

In [32]:
raw_companies= pd.read_excel('UBC_companies.xlsx')
raw2 = pd.read_excel('companysv2.xlsx')

In [33]:
companies=raw2
#companies['Papers in last 5 years']=np.nan
companies["Enviro specific papers"]=np.nan
companies.head()

,Name of Organization,Enviro specific papers
0,1920 Energy,NaN
1,88 Energy,NaN
2,AAG Energy Holdings,NaN
3,Abraxas Petroleum,NaN
4,Absolute Software,NaN


In [34]:
'''for i in range(len(companies['Name of Organization'])):
    comp=companies['Name of Organization'].iloc[i]
    #print(comp)
    companies.at[i,'Papers in last 5 years']=i #replace with query result
    companies.at[i,'Enviro specific papers']=i**2 # replace with query result'''

"for i in range(len(companies['Name of Organization'])):\n    comp=companies['Name of Organization'].iloc[i]\n    #print(comp)\n    companies.at[i,'Papers in last 5 years']=i #replace with query result\n    companies.at[i,'Enviro specific papers']=i**2 # replace with query result"

In [35]:
companies.head()

,Name of Organization,Enviro specific papers
0,1920 Energy,NaN
1,88 Energy,NaN
2,AAG Energy Holdings,NaN
3,Abraxas Petroleum,NaN
4,Absolute Software,NaN


#### Create Query

In [36]:
#Build list of queries to loop thru
# Query contains 3 main parts that change: Organization (org), Funding company (comp), and research categories
# Research categories are stored in list 'optns' for easy modification, and then constructed to the correct format in string 'cat'

queries=[] #not category specific
specqueries=[] #category specific
org = 'University of Toronto'
optns=['Environmental Sciences','Energy Fuels', 'Environmental Engineering']
for i in range(len(companies.index)):
    comp=companies['Name of Organization'].iloc[i]
    cat=str()
    for i in range(len(optns)):
        if i<len(optns)-1:
            item='('+optns[i]+')OR'
            #print(item)
        elif i==len(optns)-1:
            item='('+optns[i]+')'
            #print(item)
        cat+=item
    #print(cat)    
    specquery='OG=('+org+') AND FO=('+comp+') AND WC=('+cat+')'
    query='OG=('+org+') AND FO=('+comp+')'
    #print(query)
    specqueries.append(specquery)
    queries.append(query)


Questions/TODOs:
1. Clean company list to either only acronyms or no acronyms for entries with both. Check WoS search manually if using any combination (acronym alone, name alone, or both) provide same result? <br>
    a) does the LLC or other suffix affect search results? Need to test (use imperial oil, imperial oil Ltd, imperial oil limited, imperial oil resources, imperial oil resources ltd and imperial oil resources limited)
2. Do we want double counting prevention?
3. Need a couple manual searches to verify correct data is being pulled
4. What categories are considered enviro? Chemistry? Forestry? Enviro sci? Enviro eng?

In [37]:
url = "https://wos-api.clarivate.com/api/wos"
key = "ab99da06e236c7e3b31e3d488a95046789651be7"
headers = {'accept': 'application/json', 'X-ApiKey': key }
h2={'X-ApiKey': key}
auth = HTTPBasicAuth('X-ApiKey',key)
# loop to search, changes data sent to api each time
data= { 'databaseId': 'WOS', 'lang': 'en', 'usrQuery': 'OG=(University of Toronto) AND FO=(Imperial Oil)', 'edition': '', 'publishTimeSpan': '2018-01-01+2023-01-01', 'loadTimeSpan': '', 'createdTimeSpan': '', 'modifiedTimeSpan': '', 'count': 100, 'firstRecord': 1, 'sortField': '', 'viewField': 'WOS+UID+titles', 'optionView': 'FS', 'optionOther': '' }
for i in tqdm(range(len(companies.index))):
    data['usrQuery']=specqueries[i]
    rawdata=requests.post(url,headers=headers,json=data)
    parsed_data= json.loads(rawdata.content)
    status=rawdata.status_code
    while status==429:
        time.sleep(0.5)
        rawdata=requests.post(url,headers=headers,json=data)
        parsed_data= json.loads(rawdata.content)
        status=rawdata.status_code

    #print(rawdata.status_code)
    companies.at[i,'Papers in last 5 years']=parsed_data['QueryResult']['RecordsFound']
    #time.sleep(0.20)

100%|██████████| 1130/1130 [09:55<00:00,  1.90it/s]


In [38]:
companies.head()
companies.to_csv('testing3.csv')

## Data API exploration

In [39]:
url = "https://wos-api.clarivate.com/api/wos"
key = "ab99da06e236c7e3b31e3d488a95046789651be7"
headers = {'accept': 'application/json', 'X-ApiKey': key }
h2={'X-ApiKey': key}
auth = HTTPBasicAuth('X-ApiKey',key)
# Baseline query searching uoft and funding company only
data= { 'databaseId': 'WOS', 'lang': 'en', 'usrQuery': 'OG=(University of Toronto) AND FO=(Absolute Software Corp.)', 'edition': '', 'publishTimeSpan': '2018-01-01+2023-01-01', 'loadTimeSpan': '', 'createdTimeSpan': '', 'modifiedTimeSpan': '', 'count': 100, 'firstRecord': 1, 'sortField': '', 'viewField': 'WOS+UID+titles', 'optionView': 'FS', 'optionOther': '' }

In [40]:
stuff=requests.post(url,headers=headers, json=data)

In [41]:
print(stuff.content)

b'{"Data":{"Records":{"records":""}},"QueryResult":{"QueryID":2234,"RecordsSearched":17435882,"RecordsFound":0}}'


In [42]:
parsed = json.loads(stuff.content)
print(type(parsed))

<class 'dict'>


In [43]:
# explore data structure
parsed.keys()
parsed['QueryResult']

{'QueryID': 2234, 'RecordsSearched': 17435882, 'RecordsFound': 0}

In [44]:
parsed['QueryResult']['RecordsFound']
# Get number of results

0